In [1]:
import numpy as np
import sklearn
import pandas as pd
import re
from keras.preprocessing import sequence
from keras.models import Sequential
import random
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
import tldextract
import tensorflow

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')
data.head()

,domain,domain_type
0,000directory.com.ar,benign
1,000webhost.com,benign
2,001fans.com,benign
3,01-telecharger.com,benign
4,010shangpu.com,benign


In [3]:
#set benign to 0 and dga to 1
data['dt'] = pd.factorize(data['domain_type'])[0]
#data.dtypes 
#(do i need to set dt to catagory? find out later)

In [32]:
data['extract'] = data['domain'].apply(lambda url: tldextract.extract(url).domain)
X = data['extract']
X

0                000directory
1                  000webhost
2                     001fans
3              01-telecharger
4                  010shangpu
                 ...         
144940    hvylvkkooemakouxtal
144941        vlrlgqdlxwrcwsq
144942       wwuueisqcagqgkow
144943    egmyiaiqyfmvaprvvjv
144944    vlziidablyhoosieraw
Name: extract, Length: 144945, dtype: object

In [5]:
# Create A Dicitonary of valid chars
unique_chars = enumerate(set(''.join(X))) 
chars_dict = dict()
for i, x in unique_chars: #index of enum starts with 0
    #print('i: ' + str(i) + '  x: ' + x)
    chars_dict[x] = i + 1 #leave 0 for padding

In [6]:
# print(list(enumerate(set(''.join(X))) ))
chars_dict

{'z': 1,
 '3': 2,
 '8': 3,
 's': 4,
 'x': 5,
 '5': 6,
 '9': 7,
 'h': 8,
 'e': 9,
 'f': 10,
 'j': 11,
 'a': 12,
 '1': 13,
 '4': 14,
 'y': 15,
 'l': 16,
 'o': 17,
 'p': 18,
 'u': 19,
 '0': 20,
 'k': 21,
 'c': 22,
 '-': 23,
 'n': 24,
 'g': 25,
 '7': 26,
 'd': 27,
 'r': 28,
 'w': 29,
 'i': 30,
 'v': 31,
 '2': 32,
 '6': 33,
 't': 34,
 'b': 35,
 'm': 36,
 '\n': 37,
 'q': 38}

In [7]:
#index 0 is also going to be a feature(padding/unknown).
max_features_num = len(chars_dict) + 1

#max length will be the max length of domain in our dataset
maxlen = np.max([len(x) for x in X])

In [8]:
maxlen

56

In [9]:
# Convert characters to int
X_in_int = []
for domain in X:
    domain_in_int = []
    for c in domain:
        domain_in_int.append(chars_dict[c])
    X_in_int.append(domain_in_int)

In [10]:
X_in_int[1]


[20, 20, 20, 29, 9, 35, 8, 17, 4, 34]

In [11]:
#pad to max length using keras... do we need to use a certain max len?
X = X_in_int
X = sequence.pad_sequences(X, maxlen=maxlen)
X.shape

(144945, 56)

In [12]:
Y = data.iloc[:, 2]
Y.head()
# len(Y)

0    0
1    0
2    0
3    0
4    0
Name: dt, dtype: int64

In [13]:
## Train Model - XGBOOST
# documentation https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/


In [14]:
seed = 11
test_size = .33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

#len(X_train)+len(X_test)

In [15]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [16]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [17]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.08%


In [18]:
## PART two 
#https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

In [19]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

C:\Users\mayur\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [20]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 1000  # The number of training iterations


#https://xgboost.readthedocs.io/en/latest/parameter.html

In [21]:
model = xgb.train(param, D_train, steps)

In [22]:


preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.9865386951300626
Recall = 0.9852824043378017
Accuracy = 0.9864525840441546


In [23]:
## test accuracy on assignment

In [24]:
test = pd.read_csv('test_data.csv')

In [25]:
### INSERT THE TLDEXTRACT HERE as well

test['extract'] = test['domain'].apply(lambda url: tldextract.extract(url).domain)
XT = test['extract']

XT_in_int = []
for domain in XT:
    domain_in_int_t = []
    for c in domain:
        domain_in_int_t.append(chars_dict[c])
    XT_in_int.append(domain_in_int_t)

#need to stick the url split library in here
    # type(x1)
# x1.head()

In [26]:
XT_in_int[1]

[18, 18, 16, 17, 19, 16, 4, 17, 4, 17, 8, 31, 36, 21, 31]

In [27]:
XT = XT_in_int
XT = sequence.pad_sequences(XT, maxlen=maxlen)
XT.shape

(500, 56)

In [28]:
DT = xgb.DMatrix(XT)
preds_test = model.predict(DT)
preds_test

array([[6.02249503e-01, 3.97750527e-01],
       [1.46231940e-03, 9.98537660e-01],
       [9.68601967e-07, 9.99999046e-01],
       [7.43200371e-06, 9.99992609e-01],
       [9.99669671e-01, 3.30308249e-04],
       [5.17877221e-01, 4.82122779e-01],
       [6.41662925e-02, 9.35833633e-01],
       [9.99856114e-01, 1.43850804e-04],
       [1.02415564e-03, 9.98975873e-01],
       [1.73389778e-01, 8.26610267e-01],
       [9.72395360e-01, 2.76045892e-02],
       [3.78094501e-06, 9.99996185e-01],
       [9.54236567e-01, 4.57633808e-02],
       [2.61650216e-02, 9.73834991e-01],
       [9.99726832e-01, 2.73203885e-04],
       [2.71799844e-02, 9.72820044e-01],
       [9.98724401e-01, 1.27560995e-03],
       [6.62600279e-01, 3.37399751e-01],
       [7.98689127e-01, 2.01310843e-01],
       [2.44677588e-02, 9.75532293e-01],
       [9.97452796e-01, 2.54720007e-03],
       [7.77717304e-08, 9.99999881e-01],
       [9.97822165e-01, 2.17785733e-03],
       [9.97636080e-01, 2.36393465e-03],
       [6.011680

In [29]:
best_preds_test = np.asarray([np.argmax(line) for line in preds_test])
best_preds_test
# test['threat']=best_preds_test

# test.dtypes

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [30]:
# change 0 to benign and 1 to dga
con = []
for i in best_preds_test:
    if best_preds_test[i] == 0:
        con.append('benign')
    else:
        con.append('dga')

In [31]:
test['threat']=con
test

test.to_csv('test_RESULTS.csv', index = False, header=True)